In [14]:
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
#Import chi2 for performing chi square test 
from sklearn.feature_selection import chi2

In [15]:
from google.colab import files
uploaded = files.upload()
data = pd.read_csv('voice.csv')


Saving voice.csv to voice (2).csv


In [16]:
data.head()


,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,0.000000,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,0.000000,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,0.000000,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,0.083878,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,0.104261,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


In [17]:
X = data[['meanfreq',	'sd',	'median',	'Q25',	'Q75',	'IQR',	'skew',	'kurt',	'sp.ent',	'sfm',	'mode',	'centroid',	'meanfun',	'minfun',	'maxfun',	'meandom',	'mindom',	'maxdom',	'dfrange',	'modindx']]
Y = data['label']

X_train,X_test,Y_train,Y_test =  train_test_split(X,Y,test_size=0.2)
print(X_train.head())

      meanfreq        sd    median  ...    maxdom   dfrange   modindx
877   0.199756  0.059357  0.228475  ...  8.085938  8.062500  0.092572
1867  0.141617  0.086008  0.159907  ...  3.562500  3.539062  0.150686
2875  0.206233  0.040099  0.197469  ...  4.335938  4.125000  0.148802
603   0.165102  0.061435  0.143542  ...  0.296875  0.289062  0.166667
150   0.105873  0.089755  0.097246  ...  0.398438  0.390625  0.067097

[5 rows x 20 columns]


In [18]:
#UNIVARIATE FEATURE SELECTION
#Create array from data values
array = data.values

#Split the data into input and target
X = array[:,0:8]
Y = array[:,8]

#We will select the features using chi square
test = SelectKBest(score_func=chi2, k=4)

#Fit the function for ranking the features by score
fit = test.fit(X_train, Y_train)

#Summarize scores numpy.set_printoptions(precision=3) 
print(fit.scores_)

#Apply the transformation on to dataset
features = fit.transform(X_train)

#Summarize selected features 
print(features[0:5,:])

mask = test.get_support() #list of booleans for selected features
new_feat = [] 
for bool, feature in zip(mask, data.columns):
 if bool:
  new_feat.append(feature)
print('The best features are:{}'.format(new_feat))

[1.46136447e+00 2.82661520e+00 1.47901384e+00 1.12229345e+01
 2.51459401e-02 2.06776331e+01 3.82797474e+01 1.27548578e+04
 1.34350187e+00 2.46862076e+01 3.32194857e+00 1.46136447e+00
 1.31186488e+01 5.02974749e-01 2.49920659e-01 3.58018785e+01
 8.03948916e+00 2.80539577e+02 2.73750843e+02 1.67184885e-01]
[[2.29125814e+00 9.07650978e+00 8.08593750e+00 8.06250000e+00]
 [1.47128118e+00 5.11592722e+00 3.56250000e+00 3.53906250e+00]
 [3.97125141e+00 2.32468017e+01 4.33593750e+00 4.12500000e+00]
 [4.40736484e+00 2.95304039e+01 2.96875000e-01 2.89062500e-01]
 [2.42676927e+01 6.78335355e+02 3.98437500e-01 3.90625000e-01]]
The best features are:['skew', 'kurt', 'maxdom', 'dfrange']


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn import feature_selection
model = LogisticRegression() 
rfe = feature_selection.RFE(model, 3)

fit = rfe.fit(X_train, Y_train)

print("Num Features: %d"% fit.n_features_)
print("Selected Features: %s"% fit.support_) 
print("Feature Ranking: %s"% fit.ranking_)

Num Features: 3
Selected Features: [False False False False False  True False False  True False False False
  True False False False False False False False]
Feature Ranking: [15  6 11  2  3  1 16 18  1 13  7 14  1 12  4 17  5  9 10  8]


In [20]:
#PCA
from sklearn.decomposition import PCA
#Feature extraction
pca = PCA(n_components=3) 
fit = pca.fit(X_train)

#Summarize components
print("Explained Variance: %s" % fit.explained_variance_ratio_)

print(fit.components_)

Explained Variance: [9.98735855e-01 1.21422007e-03 4.24809510e-05]
[[-6.97883731e-05  4.27980874e-05 -6.64587142e-05 -1.24418854e-04
  -2.51568682e-05  9.92619862e-05  3.09749201e-02  9.99467194e-01
  -4.08610716e-05  1.44901477e-04 -2.33971957e-04 -6.97883731e-05
  -5.08617212e-05 -2.93119224e-05 -1.07514204e-05 -1.19242563e-03
  -4.91737928e-05 -7.24607201e-03 -7.19689822e-03 -1.86150547e-04]
 [ 2.79643518e-03 -1.41628825e-03  2.88484549e-03  3.86009063e-03
   1.49800370e-03 -2.36208693e-03 -3.51393823e-02  1.13640279e-02
  -3.48816392e-03 -1.56228025e-02  6.10965428e-03  2.79643518e-03
   1.62800205e-03  1.07677376e-03  2.25249760e-03  8.29220968e-02
   1.81599830e-04  7.04096048e-01  7.03914449e-01 -1.27293538e-02]
 [-8.72016966e-04  3.67701186e-03  5.66127289e-04 -9.88194278e-03
   6.25088449e-03  1.61328273e-02 -9.96699546e-01  3.05343572e-02
   2.12309454e-02  4.44344393e-02  8.99555698e-03 -8.72016966e-04
  -5.55247144e-03  8.74459389e-04  3.42521050e-03  3.04619889e-02
  -1.41